<a href="https://colab.research.google.com/github/Kaiziferr/multiple_models/blob/main/1_OneVsRest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import softmax

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

El propósito de este ejercicio es implementar la estrategia One-vs-Rest para un problema de clasificación.

°°°°

The goal of this exercise is to implement the One-vs-Rest strategy for a classification problem.

# **Info**
---
**@By:** Steven Bernal

**@Nickname:** Kaiziferr

**@Git:** https://github.com/Kaiziferr

# **Config**
---


In [2]:
sns.set(style="darkgrid")
pd.set_option('display.float_format', '{:,.5f}'.format)
title_data = ''
random_seed=73
warnings.filterwarnings("ignore")

# **Data**
---


Se genero un conjunto de datos sintéticos de 150 ejemplos, con 8 características, donde 7 son informativas y una redundante, con tres categorías con la misma cantidad de registros. No se pretendía que el conjunto fuera muy difícil.

°°°°

A synthetic dataset of 150 samples was generated, with 8 features, where 7 are informative and one is redundant. It contains three categories with an equal number of records. The dataset was not intended to be very difficult.


In [3]:
X, y =make_classification(
    n_samples=150,
    n_features=8,
    n_informative=7,
    n_redundant=1,
    n_classes=3,
    random_state=random_seed
)

Se emplea un contexto donde las categorías son colores, para darles un sentido a los datos.

°°°°

A context is used where the categories represent colors, in order to give meaning to the data.





In [4]:
y = pd.Series(y).replace({0:'B',1:'R',2:'G'})

# **Data Split**
---

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    shuffle=True,
    random_state=random_seed
)

Se evidencia que el conjunto está equilibrado; la pequeña discrepancia es producto del split de los datos

°°°°

It is evident that the dataset is balanced; the small discrepancy is due to the data split.


In [6]:
y_train.value_counts()

,count
B,37
R,35
G,33


**Model**
---

Se define un DataFrame para comparar las predicciones.

°°°°

A DataFrame is defined to compare the predictions.


In [7]:
predictions = pd.DataFrame(columns=['params_ovr', 'multiple_model', 'OneVsRestClassifier'])

**params_ovr**




Se implementa una regresión logística con el parámetro de la estrategia OVR.

°°°°

A logistic regression is implemented with the OVR strategy parameter.

In [8]:
model_lr =  LogisticRegression(multi_class = 'ovr')
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
predictions['params_ovr'] = y_pred_lr

**multiple_model**

Para simular la estrategia One-vs-Rest, se dividirán las categorías en tres conjuntos, con la misma cantidad de registros, solo que por cada conjunto estará la categoría de interés y el resto se modificará con 'otros'


°°°°

To simulate the One-vs-Rest strategy, the categories will be divided into three sets, each with the same number of records. For each set, the category of interest will remain, and the others will be labeled as "other."

In [9]:
y_train_blue = y_train.apply(lambda x: 'O' if x != 'B' else 'B')
y_train_red = y_train.apply(lambda x: 'O' if x != 'R' else 'R')
y_train_green = y_train.apply(lambda x: 'O' if x != 'G' else 'G')

En la matriz se muestra cómo quedan las conversiones de cada conjunto de entrenamiento.

°°°°

The matrix shows how the conversions of each training set are displayed.

In [10]:
labels = pd.concat([y_train.replace({
    'B':'Blue',
    'R':'Red',
    'G':'Green'
}),y_train_blue,y_train_red,y_train_green],axis=1)
labels.columns = ['Original','Blue','Red','Green']
labels

,Original,Blue,Red,Green
18,Red,O,R,O
117,Blue,B,O,O
95,Green,O,O,G
114,Blue,B,O,O
69,Green,O,O,G
...,...,...,...,...
67,Red,O,R,O
136,Red,O,R,O
16,Red,O,R,O
138,Red,O,R,O


Se implementan varios modelos de regresión logística, pero enfocados en los conjuntos binarios de la clase de interés y la categoría 'otros'

°°°°

Several logistic regression models are implemented, but they are focused on the binary sets of the class of interest and the 'other' category.

Blue vs Rest

In [11]:
model_lr_blue =  LogisticRegression()
model_lr_blue.fit(X_train, y_train_blue)
y_pred_lr_blue = model_lr_blue.predict_proba(X_test)

Red vs Rest

In [12]:
model_lr_red =  LogisticRegression()
model_lr_red.fit(X_train, y_train_red)
y_pred_lr_red = model_lr_red.predict_proba(X_test)

Green vs Rest

In [13]:
model_lr_green =  LogisticRegression()
model_lr_green.fit(X_train, y_train_green)
y_pred_lr_green = model_lr_green.predict_proba(X_test)

Ahora se concatenan las predicciones de cada modelo binario para extraer la probabilidad de la clase de interés, descartando la demarcada como 'otros'.

°°°°

Now, the predictions of each binary model are concatenated to extract the probability of the class of interest, discarding the one labeled as 'other'.


La selección del orden de la columna dependerá del primer registro en entrar al modelo. Aquí hay que resaltar que el modelo asume que la clase de interés es o la primera o la segunda en entrar, según el contexto que se le configure. Por eso se seleccionó la columna cero para la categoría azul, la columna uno para la categoría roja y la columna cero para la categoría verde.


°°°°

The selection of the column order will depend on the first record that enters the model. It is important to highlight that the model assumes that the class of interest is either the first or the second to enter, depending on the context set for it. This is why column zero was selected for the blue category, column one for the red category, and column zero for the green category.


In [14]:
probabilidades = np.stack(
    [y_pred_lr_blue[:,0],
     y_pred_lr_red[:,1],
     y_pred_lr_green[:,0]],axis=1)
indices_max = pd.Series(np.argmax(probabilidades, axis=1))
indices_max_prediccion = indices_max.replace({0:'B',1:'R',2:'G'})
predictions['multiple_model'] = indices_max_prediccion

**OneVsRestClassifier**

Se emplea la función de 'OneVsRestClassifier' de scikit-learn para implementar la estrategia de forma automática. Esta función permite desarrollar la estrategia mencionada en este notebook para cualquier modelo que solo funcione con modelos binarios o que se quiera implementar esta estrategia con cualquier modelo.

°°°°

The 'OneVsRestClassifier' function from scikit-learn is used to automatically implement the strategy. This function allows the development of the strategy mentioned in this notebook for any model that only works with binary models or for implementing this strategy with any model.

In [15]:
model_lr =  LogisticRegression()
oneVsRestClassifier = OneVsRestClassifier(model_lr)
oneVsRestClassifier.fit(X_train, y_train)
oneVsRestClassifier_predict = oneVsRestClassifier.predict(X_test)
predictions['OneVsRestClassifier'] = indices_max_prediccion

se anexa las etiquetas de prueba

°°°°

The test labels are appended.





In [16]:
predictions['labels_true'] = y_test.reset_index(drop=True)

In [17]:
predictions

,params_ovr,multiple_model,OneVsRestClassifier,labels_true
0,R,R,R,R
1,R,R,R,R
2,G,G,G,G
3,R,R,R,R
4,B,B,B,B
5,G,G,G,G
6,R,R,R,R
7,G,G,G,B
8,R,R,R,R
9,B,B,B,B


Los tres métodos predicen las mismas categorías

°°°°

The three methods predict the same categories.





In [18]:
predictions[predictions['params_ovr'] != predictions['multiple_model']]

,params_ovr,multiple_model,OneVsRestClassifier,labels_true


In [19]:
predictions[predictions['params_ovr'] != predictions['OneVsRestClassifier']]

,params_ovr,multiple_model,OneVsRestClassifier,labels_true


In [20]:
predictions[predictions['multiple_model'] != predictions['OneVsRestClassifier']]

,params_ovr,multiple_model,OneVsRestClassifier,labels_true


Las estrategias de los modelos tienen 12 errores de 45 registros de prueba.

°°°°

The model strategies have 12 errors out of 45 test records.





In [21]:
errores = predictions[predictions['OneVsRestClassifier'] != predictions['labels_true']]
errores

,params_ovr,multiple_model,OneVsRestClassifier,labels_true
7,G,G,G,B
11,B,B,B,G
12,B,B,B,G
13,R,R,R,G
18,B,B,B,G
25,G,G,G,R
28,G,G,G,R
30,B,B,B,G
32,G,G,G,B
34,G,G,G,R


In [22]:
errores.shape[0], errores.shape[0]/predictions.shape[0]

(12, 0.26666666666666666)

# **Info**
---
**@By:** Steven Bernal

**@Nickname:** Kaiziferr

**@Git:** https://github.com/Kaiziferr